# Spotify listening history

In [ ]:
sample(c("Oliver", "Robin", "Mike", "Jörg", "Markus"))

In [ ]:
library(tidyverse)
library(lubridate)

## Data loading

In [ ]:
data <- tibble(js=jsonlite::read_json("spotify.json")) %>% unnest_wider(js)

In [ ]:
glimpse(data)

In [ ]:
head(data)

## Day vs Night

In [ ]:
data %>% 
    mutate(time = ymd_hms(ts, locale = "de_DE.utf8"), hour = hour(time), clock=hms::as_hms(time)) %>% 
    select(ts, time, hour, clock) %>% 
    ggplot(aes(x = clock)) + geom_density()
    #identity

In [ ]:
data %>% 
    mutate(time = ymd_hms(ts, locale = "de_DE.utf8"), hour = hour(time), clock=hms::as_hms(time)) %>% 
    select(ts, time, hour, clock) %>% 
    ggplot(aes(x = clock)) + geom_density() + facet_wrap(~year(time))
    #identity

In [ ]:
data %>% 
    mutate(time = ymd_hms(ts),
           hour = hour(time), clock=hms::as_hms(time),
           timeInGermany = with_tz(time, tzone = "Europe/Berlin")) %>% 
ggplot(aes(x = hms::as_hms(timeInGermany))) + geom_density() + 
    geom_vline(xintercept = hms("14:00:00")) +
    geom_vline(xintercept = hms("17:00:00")) +
    geom_vline(xintercept = hms("21:00:00")) +
    geom_vline(xintercept = hms("3:00:00"))


In [ ]:
prepared_data <- data %>% filter(reason_end=="endplay" | reason_end=="trackdone") %>%
    mutate(time = ymd_hms(ts),
           timeInGermany = with_tz(time, tzone = "Europe/Berlin"),  
            clock=hms::as_hms(timeInGermany), 
          hour = hour(timeInGermany), 
          daymusic = hour>=14 & hour<=17,
          nightmusic = hour>=21 | hour<=3,
    )

In [ ]:
prepared_data %>% count(master_metadata_track_name, daymusic, nightmusic) %>%
filter(daymusic|nightmusic) %>% group_by(master_metadata_track_name) %>%
filter(n()<2) %>% ungroup %>% count(nightmusic)

In [ ]:
prepared_data %>% count(master_metadata_track_name, daymusic, nightmusic) %>%
filter(daymusic|nightmusic) %>% group_by(master_metadata_track_name) %>%
filter(n()>1) %>% dim()

In [ ]:
prepared_data %>% count(daymusic, nightmusic)

In [ ]:
intersection_songs = prepared_data %>% count(master_metadata_track_name, daymusic, nightmusic) %>%
filter(daymusic|nightmusic) %>% group_by(master_metadata_track_name) %>%
filter(n()>1) %>% pull(master_metadata_track_name) %>% unique

In [ ]:
prepared_data %>% filter(master_metadata_track_name %in% intersection_songs) %>%
    count(daymusic, nightmusic)